In [14]:
# ====================================================
# Character Recognition with Lightweight CNN (TF/Keras)
# ====================================================

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from collections import defaultdict

In [15]:
# -----------------------
# CONFIG
# -----------------------
train_dir = "/kaggle/input/ocr-data/OCR_data/train_data"  
test_dir  = "/kaggle/input/ocr-data/OCR_data/test_data"   
img_size = (128, 128)
batch_size = 16
epochs = 50 

In [16]:
# -----------------------
# CLASS MAPPING
# -----------------------
train_class_names = sorted({f[0] for f in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, f))})
print("Train classes (first letters):", train_class_names)

class_to_index = {cls: idx for idx, cls in enumerate(train_class_names)}
index_to_class = {idx: cls for cls, idx in class_to_index.items()}
num_classes = len(class_to_index)

print("Class mapping:", class_to_index)
print("Number of classes:", num_classes)

Train classes (first letters): ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Class mapping: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16, 'H': 17, 'I': 18, 'J': 19, 'K': 20, 'L': 21, 'M': 22, 'N': 23, 'O': 24, 'P': 25, 'Q': 26, 'R': 27, 'S': 28, 'T': 29, 'U': 30, 'V': 31, 'W': 32, 'X': 33, 'Y': 34, 'Z': 35, 'a': 36, 'b': 37, 'c': 38, 'd': 39, 'e': 40, 'f': 41, 'g': 42, 'h': 43, 'i': 44, 'j': 45, 'k': 46, 'l': 47, 'm': 48, 'n': 49, 'o': 50, 'p': 51, 'q': 52, 'r': 53, 's': 54, 't': 55, 'u': 56, 'v': 57, 'w': 58, 'x': 59, 'y': 60, 'z': 61}
Number of classes: 62


In [17]:
# -----------------------
# DATASET LOADER (Pure TF version, no py_function)
# -----------------------
keys_tensor = tf.constant(list(class_to_index.keys()))
vals_tensor = tf.constant(list(class_to_index.values()), dtype=tf.int32)
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor),
    default_value=-1
)

def process_path(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=1)   # grayscale
    img = tf.image.resize(img, img_size)
    img = (tf.cast(img, tf.float32) / 127.5) - 1.0  # normalize [-1,1]

    parts = tf.strings.split(path, os.sep)
    folder_name = parts[-2]
    label_char = tf.strings.substr(folder_name, 0, 1)
    label = table.lookup(label_char)

    return img, label

train_files = tf.data.Dataset.list_files(train_dir + "/*/*", shuffle=True)
test_files  = tf.data.Dataset.list_files(test_dir + "/*/*", shuffle=False)

full_train_ds = train_files.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

# Split into train/validation (e.g. 80/20)
train_size = int(0.8 * len(list(train_files)))
train_ds = (full_train_ds.take(train_size)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE))
val_ds = (full_train_ds.skip(train_size)
          .batch(batch_size)
          .prefetch(tf.data.AUTOTUNE))

test_ds = (test_files
           .map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(batch_size)
           .prefetch(tf.data.AUTOTUNE))

In [18]:
# -----------------------
# LIGHTWEIGHT CNN MODEL
# -----------------------
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation="relu", padding="same", 
                  kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                  input_shape=(128,128,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(32, (3,3), activation="relu", padding="same", 
                  kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation="relu", padding="same", 
                  kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 62)             │         7,998 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,128,574 (8.12 MB)

 Trainable params: 2,128,574 (8.12 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# -----------------------
# COMPILE & TRAIN
# -----------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=[early_stop]
)


Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 135ms/step - accuracy: 0.0135 - loss: 4.2556 - val_accuracy: 0.0645 - val_loss: 4.0733
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 122ms/step - accuracy: 0.0653 - loss: 3.9735 - val_accuracy: 0.1613 - val_loss: 3.4777
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 131ms/step - accuracy: 0.1450 - loss: 3.5288 - val_accuracy: 0.4355 - val_loss: 2.9236
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - accuracy: 0.2374 - loss: 2.9347 - val_accuracy: 0.6290 - val_loss: 1.7277
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 123ms/step - accuracy: 0.3556 - loss: 2.3466 - val_accuracy: 0.8065 - val_loss: 1.2399
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - accuracy: 0.4152 - loss: 2.1118 - val_accuracy: 0.8145 - val_loss: 0.9517
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 124ms/step - accuracy: 0.4953 - loss: 1.7415 - val_accuracy: 0.9516 - val_loss: 0.4320
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 126ms/step - accuracy: 0.6002 - loss: 1.4075 - val_accuracy: 0.

In [20]:
# -----------------------
# SAVE MODEL
# -----------------------
model.save("lightweight_charcnn_tf.h5")
print("Model saved!")

Model saved!


In [21]:
# -----------------------
# LOAD MODEL
# -----------------------
loaded_model = tf.keras.models.load_model("lightweight_charcnn_tf.h5")
print("Model loaded!")

Model loaded!


In [22]:
# -----------------------
# EVALUATE ON TEST DATA
# -----------------------
correct = 0
wrong = 0
total = 0

y_true = []
y_pred = []
class_correct = defaultdict(int)
class_total = defaultdict(int)

for images, labels in test_ds:
    preds = loaded_model.predict(images, verbose=0)
    predicted_classes = np.argmax(preds, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(predicted_classes)

    correct += np.sum(predicted_classes == labels.numpy())
    wrong += np.sum(predicted_classes != labels.numpy())
    total += labels.shape[0]

    for true, pred in zip(labels.numpy(), predicted_classes):
        class_total[true] += 1
        if true == pred:
            class_correct[true] += 1

print("\n=== Per-Class Results ===")
for idx in sorted(class_total.keys()):
    total_i = class_total[idx]
    correct_i = class_correct[idx]
    wrong_i = total_i - correct_i
    print(f"Class {index_to_class[idx]}: Correct={correct_i}, Wrong={wrong_i}, Total={total_i}, Acc={100*correct_i/total_i:.2f}%")

print("\n=== Overall Results ===")
print(f"Correct predictions: {correct}")
print(f"Wrong predictions: {wrong}")
print(f"Total images: {total}")
print(f"Accuracy: {100.0 * correct / total:.2f}%")

precision = precision_score(y_true, y_pred, average="macro")
recall = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")

print("\n=== Precision / Recall / F1 (Macro) ===")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=[index_to_class[i] for i in range(num_classes)]))



=== Per-Class Results ===
Class 0: Correct=10, Wrong=15, Total=25, Acc=40.00%
Class 1: Correct=0, Wrong=25, Total=25, Acc=0.00%
Class 2: Correct=3, Wrong=22, Total=25, Acc=12.00%
Class 3: Correct=5, Wrong=20, Total=25, Acc=20.00%
Class 4: Correct=14, Wrong=11, Total=25, Acc=56.00%
Class 5: Correct=9, Wrong=16, Total=25, Acc=36.00%
Class 6: Correct=16, Wrong=9, Total=25, Acc=64.00%
Class 7: Correct=6, Wrong=19, Total=25, Acc=24.00%
Class 8: Correct=10, Wrong=15, Total=25, Acc=40.00%
Class 9: Correct=7, Wrong=18, Total=25, Acc=28.00%
Class A: Correct=10, Wrong=15, Total=25, Acc=40.00%
Class B: Correct=14, Wrong=11, Total=25, Acc=56.00%
Class C: Correct=8, Wrong=17, Total=25, Acc=32.00%
Class D: Correct=17, Wrong=8, Total=25, Acc=68.00%
Class E: Correct=17, Wrong=8, Total=25, Acc=68.00%
Class F: Correct=15, Wrong=10, Total=25, Acc=60.00%
Class G: Correct=8, Wrong=17, Total=25, Acc=32.00%
Class H: Correct=19, Wrong=6, Total=25, Acc=76.00%
Class I: Correct=16, Wrong=9, Total=25, Acc=64.00%